# Deploy the PowerSkill to Azure Search

In [5]:
import os
from requests import post, put
from dotenv import load_dotenv

load_dotenv(dotenv_path='.env')

API_KEY             = os.getenv("SEARCH_API_KEY")
ACS_URL             = os.getenv("ACS_URL")
DATA_SOURCE         = os.getenv("DATA_SOURCE")
CONTAINER_NAME      = os.getenv("STORAGE_CONTAINER_NAME")
CONNECTION_STRING   = os.getenv("STORAGE_CONNECTION_STRING")
INDEX_NAME          = os.getenv("INDEX_NAME")
SKILLSET_NAME       = os.getenv("SKILLSET_NAME")
COGSVC_KEY          = os.getenv("COGSVC_KEY")
WEB_APP_KEY         = os.getenv("WEB_APP_KEY")
WEB_APP_URL         = os.getenv("WEB_APP_URL")

print (API_KEY)

3F2715C5FE3281BB9B43FE74972A39A7


## Create the data source

In [2]:
json_text = {
    "name" : DATA_SOURCE,
    "type" : "azureblob",
    "credentials" : { "connectionString" : CONNECTION_STRING },    #  This is your azure blob connection string
    "container" : { "name" : CONTAINER_NAME }  # The name of the container where the data files are
}

headers = {
    "api-key": API_KEY,     
    "Content-Type": "application/json"
}

try:
    url = f"{ACS_URL}/datasources?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
        
        
    if resp.status_code == 201:
        print("Success creating data source")
        
except Exception as e:
    print('Exception creating data source', e)

## Now we create the index

In [3]:
json_text = {
      "name" : INDEX_NAME,
      "fields": [
        { "name": "id", "type": "Edm.String", "key": True, "searchable": False },
        { "name": "file_name", "type": "Edm.String", "searchable": False },
        { "name": "size", "type": "Edm.Int64", "searchable": False },
        { "name": "last_modified", "type": "Edm.DateTimeOffset", "searchable": False },
        { "name": "content", "type": "Edm.String", "searchable": True, "filterable": False, "sortable": False, "facetable": False },
        { "name": "summary", "type": "Edm.String", "searchable": True, "filterable": False, "sortable": False, "facetable": False },
        { "analyzer": "standard.lucene", "facetable": False, "filterable": False, "indexAnalyzer": None, "key": False, 
            "name": "organization",
            "retrievable": True,
            "searchAnalyzer": None,
            "searchable": True,
            "sortable": False,
            "synonymMaps": [],
            "type": "Collection(Edm.String)"
        },
        {
            "analyzer": "standard.lucene",
            "facetable": False,
            "filterable": False,
            "indexAnalyzer": None,
            "key": False,
            "name": "person",
            "retrievable": True,
            "searchAnalyzer": None,
            "searchable": True,
            "sortable": False,
            "synonymMaps": [],
            "type": "Collection(Edm.String)"
        },
        {
            "analyzer": "standard.lucene",
            "facetable": False,
            "filterable": False,
            "indexAnalyzer": None,
            "key": False,
            "name": "location",
            "retrievable": True,
            "searchAnalyzer": None,
            "searchable": True,
            "sortable": False,
            "synonymMaps": [],
            "type": "Collection(Edm.String)"
        },
        {
            "analyzer": "standard.lucene",
            "facetable": False,
            "filterable": False,
            "indexAnalyzer": None,
            "key": False,
            "name": "url",
            "retrievable": True,
            "searchAnalyzer": None,
            "searchable": True,
            "sortable": False,
            "synonymMaps": [],
            "type": "Collection(Edm.String)"
        }
      ]
}

try:
    url = f"{ACS_URL}/indexes?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
        
    if resp.status_code == 400:
        print(f"Error", resp.text)    
            
    if resp.status_code == 201:
        print("Success creating index")
        
except Exception as e:
    print('Exception creating index', e)

Error {"error":{"code":"ResourceNameAlreadyInUse","message":"Cannot create index 'text-summarization-index' because it already exists.","details":[{"code":"CannotCreateExistingIndex","message":"Cannot create index 'text-summarization-index' because it already exists."}]}}


## Now we create the skill set

In [7]:
json_text = {
    "description": "Crack documents.",
    "skills": [
        {
          "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
          "categories": [ "Organization", "Location", "Person", "Url" ],
          "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text", "source": "/document/content"
            }
          ],
          "outputs": [
            {
              "name": "organizations", "targetName": "organizations"
            },
            {
              "name": "persons", "targetName": "persons"
            },
            {
              "name": "locations", "targetName": "locations"
            },
            {
              "name": "urls", "targetName": "urls"
            }
          ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "A custom skill that summarizes the text",
            "uri": f"{WEB_APP_URL}/api/extraction",
            "timeout": "PT160S",
            "batchSize": 1,
            "context": "/document/content",
            "httpHeaders": {
                "Ocp-Apim-Subscription-Key": WEB_APP_KEY
            },
            "httpMethod": "POST",
            "inputs": [
            {
                "name": "text",
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "summary"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "cogsvc",
        "key": COGSVC_KEY
    }
}



try:
    url = f"{ACS_URL}/skillsets/{SKILLSET_NAME}?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
    
    if resp.status_code == 400:
        print(f"Error", resp.text)

    if resp.status_code == 201:
        print("Success creating skillset")
        
except Exception as e:
    print('Exception creating skillset', e)


## Now we create the indexer

In [8]:
json_text = {
    "name": "text-summarization-indexer",
    "dataSourceName": DATA_SOURCE,
    "targetIndexName": INDEX_NAME,
    "skillsetName": SKILLSET_NAME,
    "parameters": {
        "configuration": {
            "allowSkillsetToReadFileData": True
        }
    },
    "outputFieldMappings": [
      {
        "sourceFieldName" : "/document/organizations",
        "targetFieldName" : "organization"
      },
      {
        "sourceFieldName" : "/document/persons",
        "targetFieldName" : "person"
      },
      {
        "sourceFieldName" : "/document/locations",
        "targetFieldName" : "location"
      },
      {
        "sourceFieldName" : "/document/urls",
        "targetFieldName" : "url"
      },
      {
        "sourceFieldName": "/document/content/summary",
        "targetFieldName": "summary"
      }
    ]
}


try:
    url = f"{ACS_URL}/indexers?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")

    if resp.status_code == 400:
        print(f"Error", resp.text)
        
    if resp.status_code == 201:
        print("Success creating indexer")
        
except Exception as e:
    print('Exception creating indexer', e)

Error {"error":{"code":"","message":"Cannot create indexer 'text-summarization-indexer' because an indexer with that name already exists"}}
